In [45]:
module Society
    export SocietyType

    mutable struct SocietyType
        population::Int              # エージェントの初期数
        sex::Vector{AbstractString}  # エージェントの性別
        age::Vector{Int}             # エージェントの年齢

        # コンストラクタの定義
        SocietyType(firstsize) = new(
            firstsize,
            ["m" for i = 1:firstsize],  # とりあえず最初は全部オスにしとく
            [0 for i = 1:firstsize]     # とりあえず最初は全部0歳にしとく
        )
    end
end

Main.Society

In [46]:
module Decision
    export choose_init_males, initialize_sex, initialize_age, copulation, aging, death, population_control, calculation
    using StatsBase

    
    # 初期人口から初期オス比の分を、オスのエージェントとしてランダムサンプリングする
    function choose_init_males(firstsize, init_male_prop)
        init_male = StatsBase.self_avoid_sample!(1:firstsize, [i for i = 1:Int(firstsize * init_male_prop)])

        return init_male
    end


    # 全エージェントの性別を初期化
    function initialize_sex(society, init_male)
        for id = 1:society.population
            society.strategy[id] = ifelse(id in init_male, "m", "f")  # ifelse(条件式, Trueの時の処理, Falseの時の処理)
        end
    end


    # 全エージェントの年齢を初期化
    function initialize_age(society, life_length)
        society.age = [rand(0:(life_length - 1)) for i = 1:society.population]
    end


    # 集団内の交尾
    function copulation(society, cool_time, born_rate)
        # 生殖可能なオス個体のインデックス
        male_copu_id = [i for i = 1:society.population if (society.sex[i] == "m") && (society.age[i] > cool_time)]

        # 生殖可能なメス個体のインデックス
        female_copu_id = [i for i = 1:society.population if (society.sex[i] == "f") && (society.age[i] > cool_time)]

        # 流産しない個体を選ぶ
        chosen_female_id = [female_copu_id[i] for i = eachindex(female_copu_id) if rand() <= born_rate]

        # メスが子供を産む
        if length(male_copu_id) != 0
            for i = 1:length(chosen_female_id)
                push!(society.sex, sample(["m", "f"]))  # 子供の性別は "m":"f" = 1:1
                push!(society.age, 0)                   # 子供の年齢は0歳
                society.population += 1
            end
        end
    end


    # 歳をとる
    function aging(society)
        society.age = broadcast(+, society, 1)  # ブロードキャストで足し算
    end


    # 個体が死亡する
    function death(society, life_length, accident_rate)
        # 死亡する個体のインデックスを記録
        death_list = []
        for i = 1:society.population
            if society.age[i] >= life_length
                push!(death_list, i)         # 寿命で死亡する個体のindex
            elseif rand() <= ((1/life_length) + accident_rate)
                push!(death_list, i)         # 一定の死亡率で死亡する個体のindex
            end
        end

        # 個体を取り除く
        deleteat!(society.age, death_list)
        deleteat!(society.sex, death_list)
        society.population -= length(death_list)
    end


    # 個体数抑制
    function population_control(society, Nmax)
        while length(society.population) > Nmax
            index = rand(1:society.population)
            deleteat!(society.age, index)
            deleteat!(society.sex, index)
            society.population -= 1
        end
    end


    # 個体数、性比を記録する
    function calculation(society)
        population_size = society.population
        male_size = length([1 for sex = society.sex if sex == "m"])
        male_proportion = ifelse(population_size != 0, male_size/population_size, 0)

        return populationsize, male_proportion
    end
end

Main.Decision

In [47]:
module Simulation
    export run_one_episode, main
    using ..Society
    using ..Decision
    using CSV
    using DataFrames
    using Random

    # シミュレーション実行
    function run_one_episode(society, iteration, firstsize, Nmax, life_length, cool_time, init_male_prop, accident_rate, born_rate, episode, seeds)
        DataFrame(step = [], population_size = [], male_proportion = []) |> CSV.write("./results/result$(seeds)/episode_$(episode).csv")

        # 一つの時間軸でシミュレーションを実行
        init_male = choose_init_males(firstsize, init_male_prop)
        initialize_sex(society, init_male)
        initialize_age(society, life_length)
        for step = 1:iteration
            copulation(society, cool_time, born_rate)
            aging(society)
            death(society, life_length, accident_rate)
            population_control(society, Nmax)
            global popu_size, male_prop = calculation(society)

            DataFrame(step = [step], population_size = [popu_size], male_proportion = [male_prop]) |> CSV.write("./results/result$(seeds)/episode_$(episode).csv", append=true)
        end
    end


    # main処理
    # 死亡割合、流産しない割合を変化させて最終個体数の変化を観察
    # 色々なseedでアンサンブル平均を取る
    function main(iteration, firstsize, Nmax, life_length, cool_time, init_male_prop, accident_rate_array, born_rate_array, episode, seeds)

        society = SocietyType(firstsize)

        for seeds = 1:length(seeds)
            Random.seed(seeds)
            mkpath("./results/result$(seeds)")

            # パラメータの組み合わせを変えシミュレーション実行
            DataFrame(episode = [], accident_rate = [], born_rate = []) |> CSV.write("./results/parameters.csv")
            for i = eachindex(accident_rate_array)
                for j = eachindex(born_rate_array)
                    episode = (length(accident_rate_array) * (i-1)) + j
                    accident_rate = accident_rate_array[i]
                    born_rate = born_rate_array[j]
                    run_one_episode(society, iteration, firstsize, Nmax, life_length, cool_time, init_male_prop, accident_rate, born_rate, episode, seeds)
                    DataFrame(episode = episode, accident_rate = accident_rate, born_rate = born_rate) |> CSV.write("./results/parameters.csv", append=true)
                end
            end
            println("seed$(seeds):done")
        end
    end
end


# 実行部分
using .Simulation

# パラメータの設定
const iteration = 50
const firstsize = 100
const Nmax = 10000
const life_length = 30
const cool_time = 10
const init_male_prop = 0.5
const accident_rate_array = Vector(0:0.02:0.2) # [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2]
const born_rate_array = Vector(0:0.1:1)        # [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
const seeds = Vector(1:100)

# パラメータをテキストファイルで保存する
mkdir("./results")
open("./results/params.txt", "w") do out
    println(out, "iteration = $(iteration)")
    println(out, "firstsize = $(firstsize)")
    println(out, "Nmax = $(Nmax)")
    println(out, "life_length = $(life_length)")
    println(out, "cooltime = $(cool_time)")
    println(out, "init_male_prop = $(init_male_prop)")
    println(out, "accident_rate_array = $(accident_rate_array)")
    println(out, "born_rate_array = $(born_rate_array)")
    println(out, "seeds = $(seeds)")
end

@time main(iteration, firstsize, Nmax, life_length, cool_time, init_male_prop, accident_rate_array, born_rate_array, episode, seeds)

LoadError: UndefVarError: main not defined

In [2]:
r = Vector(2:11)
for i = eachindex(r)
    println(i)
end

1
2
3
4
5
6
7
8
9
10


In [10]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9]
deleteat!(a, [1, 3, 5])
println(a)

[2, 4, 6, 7, 8, 9]
